# Will Bank's Clients subscribe to term deposit?

Data set comes from https://www.kaggle.com/ishandutta/bank-marketing-data-set.
This report is an extension of previous one called 'Classification Project- Comparing Different Methods', which can be found here: https://github.com/hannia941/Classification.

In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

### Preprocessing the data

This part is the same as in the last project (mentioned at the top). For more details please go to https://github.com/hannia941/Classification
1. Loading the dataset
2. Encoding categorical data
3. Spliting data into training and testing set
4. Feature Scaling
No missing data was found.

In [2]:
#1. Loading the dataset
bank_data = pd.read_csv('/Users/user/Desktop/Python/Classification Project/bank-additional-full.csv',sep = ';')
bank_data.columns = ['age', 'job_type', 'marital', 'education', 'has_credit', 'has_house_loan', 'has_person_loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pass_days',
       'previous', 'prev_outcome', 'emp_var_rate', 'cpi',
       'cci', 'euribor3m', 'nr_employed', 'class']
bank_data_min = bank_data[['job_type', 'marital', 'education', 'has_credit', 'has_house_loan',
                          'has_person_loan','contact', 'prev_outcome', 'age', 'campaign', 'previous',
                          'emp_var_rate', 'cpi','cci', 'euribor3m', 'class']]

y = bank_data_min.iloc[:,-1].values
X = bank_data_min.iloc[:,:-1].values

#2. Encoding categorical data
X_b4_encode = bank_data_min.iloc[:,:-1].values
ohe = OneHotEncoder()
col_trans = ColumnTransformer(transformers= [('encoder',ohe,[0,1,2,3,4,5,6,7])])
X_b4_encode = col_trans.fit_transform(X_b4_encode)
X_b4_encode = pd.DataFrame(X_b4_encode.toarray())
name_list = [string.replace("encoder__","") for string in col_trans.get_feature_names()]
var_encoder = {'x0': 'job_type', 'x1': 'marital', 'x2': 'education', 'x3': 'has_credit', 'x4': 'has_house_loan', \
               'x5': 'has_person_loan', 'x6': 'contact', 'x7': 'prev_outcome'}
name_list = [string.replace(string[:2],var_encoder[string[:2]]) for string in name_list]
X_b4_encode.columns = name_list
X = pd.concat([X_b4_encode,pd.DataFrame(X[:,8:])],axis=1)
name_list.extend(['age', 'campaign', 'previous', 'emp_var_rate', 'cpi','cci', 'euribor3m'])
X.columns = name_list

le = LabelEncoder()
y = le.fit_transform(y)

#3. Spliting data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#4. Feature Scaling
sc = StandardScaler()
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train[:,38:] = sc.fit_transform(X_train[:,38:])
X_test[:,38:] = sc.transform(X_test[:,38:])

### Feature Selection

The approach is to train dataset using a Decision Tree and use obtained ranking of importance of the features to train ANN quicker.

In [3]:
clf = ExtraTreesClassifier()
clf = clf.fit(X_train, y_train)
model = SelectFromModel(clf, prefit=True)
X_train_reduced = model.transform(X_train)
feature_idx = model.get_support()
feature_name = X.columns[feature_idx]
X_test_reduced = X_test[:,feature_idx]

In [4]:
len(feature_name)
#Number of features we will be building the network with

7

### Building Artificial Neural Network

Since there is no hard evidence or test to perform in order to find the exact number of layers and nodes for the ANN and there are only guidelines of how to do it, I will be experimenting with different combinations of those numbers. 
1. two hidden layers with 5 nodes each
2. two hidden layers with 6 nodes and 4 nodes
3. two hidden layers with 3 nodes each
4. hidden layer with 6 nodes
5. hidden layer with 4 nodes
Accuracy will be checked using Confusion Matrix. For more elaborate accuracy rates please take a look at my previous project, that was mentioned at the very top.
Activation function for hidden layer is a rectifier function, for output layer- sigmoid.
Those functions are most common to use for binary dependent variable. 

In [5]:
classifierANN_1 = Sequential()
classifierANN_2 = Sequential()
classifierANN_3 = Sequential()
classifierANN_4 = Sequential()
classifierANN_5 = Sequential()

#### Classifier 1.

In [6]:
#Input layer and 1st hidden layer
classifierANN_1.add(Dense(5, activation = 'relu'))
#2nd hidden layer
classifierANN_1.add(Dense(5, activation = 'relu'))
#Output layer
classifierANN_1.add(Dense(1, activation = 'sigmoid'))
#Compilion the ANN
classifierANN_1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Fitting classifier to the training set 
classifierANN_1.fit(X_train_reduced, y_train, batch_size=32, epochs=100)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
32950/32950 [==============================] - 2s 49us/sample - loss: 0.4555 - acc: 0.7988
Epoch 2/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2947 - acc: 0.8893
Epoch 3/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2875 - acc: 0.89520s - loss: 0.2877 - ETA: 0s - loss: 0.2863 - acc: 0.8
Epoch 4/100
32950/32950 [==============================] - 2s 47us/sample - loss: 0.2854 - acc: 0.8973
Epoch 5/100
32950/32950 [==============================] - 2s 50us/sample - loss: 0.2845 - acc: 0.89921s - loss - ETA: 0s - loss: 0.2843 - acc: 0
Epoch 6/100
32950/32950 [==============================] - 1s 38us/sample - loss: 0.2842 - acc: 0.8995
Epoch 7/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2837 - acc: 0.8995
Epoch 8/100
32950/32950 [==============================] - 1s 3

In [7]:
#Predicting y value
y_pred = classifierANN_1.predict(X_test_reduced)
y_pred = (y_pred>0.5)

#Checking accuracy score
cm = confusion_matrix(y_test,y_pred)
acc_1 = accuracy_score(y_test,y_pred)

#### Classifier 2.

In [8]:
#Input layer and 1st hidden layer
classifierANN_2.add(Dense(6, activation = 'relu'))
#2nd hidden layer
classifierANN_2.add(Dense(4, activation = 'relu'))
#Output layer
classifierANN_2.add(Dense(1, activation = 'sigmoid'))
#Compilion the ANN
classifierANN_2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Fitting classifier to the training set 
classifierANN_2.fit(X_train_reduced, y_train, batch_size=32, epochs=100)

Epoch 1/100
32950/32950 [==============================] - 1s 45us/sample - loss: 0.3654 - acc: 0.8775
Epoch 2/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2941 - acc: 0.88781s - loss: 0.282 - ETA: 0s - loss: 0.2934 - acc:
Epoch 3/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2918 - acc: 0.8878
Epoch 4/100
32950/32950 [==============================] - 2s 50us/sample - loss: 0.2900 - acc: 0.8878
Epoch 5/100
32950/32950 [==============================] - 1s 45us/sample - loss: 0.2884 - acc: 0.8934
Epoch 6/100
32950/32950 [==============================] - 2s 46us/sample - loss: 0.2870 - acc: 0.8983
Epoch 7/100
32950/32950 [==============================] - 2s 51us/sample - loss: 0.2857 - acc: 0.8985
Epoch 8/100
32950/32950 [==============================] - 1s 44us/sample - loss: 0.2844 - acc: 0.89860s - loss: 0.2747 - acc - ETA: 0s - loss: 0.2835 - ac
Epoch 9/100
32950/32950 [==============================] - 1s 43us/sample -

In [9]:
#Predicting y value
y_pred = classifierANN_2.predict(X_test_reduced)
y_pred = (y_pred>0.5)

#Checking accuracy score
cm = confusion_matrix(y_test,y_pred)
acc_2 = accuracy_score(y_test,y_pred)

#### Classifier 3.

In [10]:
#Input layer and 1st hidden layer
classifierANN_3.add(Dense(3, activation = 'relu'))
#2nd hidden layer
classifierANN_3.add(Dense(3, activation = 'relu'))
#Output layer
classifierANN_3.add(Dense(1, activation = 'sigmoid'))
#Compilion the ANN
classifierANN_3.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Fitting classifier to the training set 
classifierANN_3.fit(X_train_reduced, y_train, batch_size=32, epochs=100)

Epoch 1/100
32950/32950 [==============================] - 2s 53us/sample - loss: 0.4325 - acc: 0.8646
Epoch 2/100
32950/32950 [==============================] - 1s 43us/sample - loss: 0.3135 - acc: 0.8878
Epoch 3/100
32950/32950 [==============================] - 2s 46us/sample - loss: 0.2998 - acc: 0.8878
Epoch 4/100
32950/32950 [==============================] - 1s 40us/sample - loss: 0.2950 - acc: 0.88780s - loss: 0.2954 - acc: 0.88
Epoch 5/100
32950/32950 [==============================] - 1s 40us/sample - loss: 0.2934 - acc: 0.88780s - loss: 0.30
Epoch 6/100
32950/32950 [==============================] - 1s 44us/sample - loss: 0.2921 - acc: 0.8878
Epoch 7/100
32950/32950 [==============================] - 1s 39us/sample - loss: 0.2912 - acc: 0.8878
Epoch 8/100
32950/32950 [==============================] - 1s 40us/sample - loss: 0.2906 - acc: 0.8888
Epoch 9/100
32950/32950 [==============================] - 1s 39us/sample - loss: 0.2901 - acc: 0.8909
Epoch 10/100
32950/32950 [===

In [11]:
#Predicting y value
y_pred = classifierANN_3.predict(X_test_reduced)
y_pred = (y_pred>0.5)

#Checking accuracy score
cm = confusion_matrix(y_test,y_pred)
acc_3 = accuracy_score(y_test,y_pred)

#### Classifier 4.

In [12]:
#Input layer and 1st hidden layer
classifierANN_4.add(Dense(6, activation = 'relu'))
#Output layer
classifierANN_4.add(Dense(1, activation = 'sigmoid'))
#Compilion the ANN
classifierANN_4.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Fitting classifier to the training set 
classifierANN_4.fit(X_train_reduced, y_train, batch_size=32, epochs=100)

Epoch 1/100
32950/32950 [==============================] - 1s 43us/sample - loss: 0.4207 - acc: 0.82361s - loss: 0.
Epoch 2/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2915 - acc: 0.89560s - loss: 0.2913 - acc: 0.895 - ETA: 0s - loss: 0.2926 - acc: 
Epoch 3/100
32950/32950 [==============================] - 2s 55us/sample - loss: 0.2877 - acc: 0.89660s - loss: 0.2900 - acc: 
Epoch 4/100
32950/32950 [==============================] - 1s 45us/sample - loss: 0.2860 - acc: 0.89901s - loss: 0
Epoch 5/100
32950/32950 [==============================] - 1s 38us/sample - loss: 0.2849 - acc: 0.89910s - loss: 0.2907  - ETA: 0s - loss: 0.2859 - acc: 0.89 - ETA: 0s - loss: 0.2849 - acc: 0.8
Epoch 6/100
32950/32950 [==============================] - 1s 43us/sample - loss: 0.2842 - acc: 0.89920s - loss: 0.2815 - 
Epoch 7/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2837 - acc: 0.89940s - loss: 0.2860 -
Epoch 8/100
32950/32950 [===========

In [13]:
#Predicting y value
y_pred = classifierANN_4.predict(X_test_reduced)
y_pred = (y_pred>0.5)

#Checking accuracy score
cm = confusion_matrix(y_test,y_pred)
acc_4 = accuracy_score(y_test,y_pred)

#### Classifier 5.

In [14]:
#Input layer and 1st hidden layer
classifierANN_5.add(Dense(4, activation = 'relu'))
#Output layer
classifierANN_5.add(Dense(1, activation = 'sigmoid'))
#Compilion the ANN
classifierANN_5.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Fitting classifier to the training set 
classifierANN_5.fit(X_train_reduced, y_train, batch_size=32, epochs=100)

Epoch 1/100
32950/32950 [==============================] - 1s 44us/sample - loss: 0.3611 - acc: 0.8793
Epoch 2/100
32950/32950 [==============================] - 1s 38us/sample - loss: 0.2902 - acc: 0.8975
Epoch 3/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2871 - acc: 0.8992
Epoch 4/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2858 - acc: 0.89950s - loss: 0.
Epoch 5/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2851 - acc: 0.8995
Epoch 6/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2849 - acc: 0.8996
Epoch 7/100
32950/32950 [==============================] - 1s 44us/sample - loss: 0.2848 - acc: 0.89970s - loss: 0.2867 - ac
Epoch 8/100
32950/32950 [==============================] - 1s 38us/sample - loss: 0.2846 - acc: 0.8998
Epoch 9/100
32950/32950 [==============================] - 1s 37us/sample - loss: 0.2844 - acc: 0.8996
Epoch 10/100
32950/32950 [============

In [15]:
#Predicting y value
y_pred = classifierANN_5.predict(X_test_reduced)
y_pred = (y_pred>0.5)

#Checking accuracy score
cm = confusion_matrix(y_test,y_pred)
acc_5 = accuracy_score(y_test,y_pred)

In [27]:
scores = pd.DataFrame([acc_1, acc_2, acc_3, acc_4, acc_5]).transpose()
scores.columns = ['Classifier1', 'Classifier2', 'Classifier3', 'Classifier4', 'Classifier5']
scores

,Classifier1,Classifier2,Classifier3,Classifier4,Classifier5
0,0.896091,0.897062,0.896698,0.895727,0.896698


## Second classifier has the best accuracy.